In [1]:
import re
import os
from collections import defaultdict
import pandas as pd
from pathlib import Path # For data paths
import numpy as np
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

pd.set_option('display.max_rows', 500)

In [2]:
def tableDataText(table):    
    """Parses a html segment started with tag <table> followed 
    by multiple <tr> (table rows) and inner <td> (table data) tags. 
    It returns a list of rows with inner columns. 
    Accepts only one <th> (table header/data) in the first row.
    """
    def rowgetDataText(tr, coltag='td'): # td (data) or th (header)       
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]  
    rows = []
    trs = table.find_all('tr')
    headerow = rowgetDataText(trs[0], 'th')
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append(rowgetDataText(tr, 'td') ) # data row       
    return rows



In [3]:
def format_color_groups(df):
    colors = ['gold', 'lightblue']
    x = df.copy()
    factor = 0.5

    style = f'background-color: {colors[0]}'
    style2 = f'background-color: {colors[0]}'
    x.loc[x['Difference'] < factor, :] = style
        
    return x

In [4]:
headers = {'Accept-Encoding': 'identity'}
try:
    response = requests.get("https://www.borzamalta.com.mt/?handler=TradingBoard", headers=headers)
except requests.ConnectionError as error:
    print(error)

In [5]:
list_table=[]
bs = BeautifulSoup(response.content)
tables = bs.findAll('table')
for table in tables:
    df = pd.DataFrame(tableDataText(table))
    if len(df.index) > 15:
        df=df.rename(columns=df.iloc[0]).drop(df.index[0])
        list_table.append(df)

In [ ]:
list_table

Table 2 - Corporate Bonds
Table 5 - Regulated Market
Table 7 - Corporate Bonds offers

In [9]:
trades = list_table[5].set_index('Symbol Code').filter(regex='A$|B$', axis=0)
trades

,Security Name,Best Bid Count,Best Bid Volume,Best Bid Price,Best Offer Price,Best Offer Volume,Best Offer Count
Symbol Code,,,,,,,
BD27A,4.75% Best Deal Properties Holding plc 2027,1,"7,000",99.000,100.950,"7,000",1


In [39]:
offers = list_table[-3].set_index('Symbol Code')
offers = offers.replace('-', np.NaN)
offers['Best Offer Price'] = offers['Best Offer Price'].astype(float)
offers['Best Bid Count'] = offers['Best Bid Count'].astype(float)
offers['Best Bid Price'] = offers['Best Bid Price'].astype(float)
offers['Yield'] = offers['Security Name'].str.extract('((?:\d+%)|(?:\d+\.\d+%))')
offers['Maturity'] = offers['Security Name'].str.extract('(2\d\d\d)').astype(int)
offers['YTM'] = (((offers['Yield'].str.rstrip('%').astype('float')) + (100-offers['Best Offer Price'])/(offers['Maturity']-2023))*100 / ((100+offers['Best Offer Price'])/2))
offers['YTM'] = np.round(offers['YTM'],2)
offers = offers.sort_values(by=['YTM'], ascending=False)
offers

,Security Name,Best Bid Count,Best Bid Volume,Best Bid Price,Best Offer Price,Best Offer Volume,Best Offer Count,Yield,Maturity,YTM
Symbol Code,,,,,,,,,,
MT28A,4.85% Melite Finance plc 2028,NaN,NaN,NaN,90.00,"75,000",1,4.85%,2028,7.21
MS26A,4.5% MedservRegis plc 2026,NaN,NaN,NaN,95.40,"53,600",1,4.5%,2026,6.18
MS26B,5.75% MedservRegis plc 2026,1.0,"20,600",95.00,99.50,"20,000",1,5.75%,2026,5.93
GM30A,6.25% GPH Malta Finance plc 2030,1.0,"3,000",100.50,102.50,"20,000",1,6.25%,2030,5.82
PF33A,6% Pharmacare Finance plc 2033,1.0,"3,000",100.50,102.49,"10,000",1,6%,2033,5.68
BV30B,3.5% Bank Of Valletta plc 2030,1.0,"8,000",87.00,87.49,"10,000",1,3.5%,2030,5.64
TG26A,5.9% Together Gaming Solutions plc 2026,1.0,"2,000",100.00,100.90,"6,000",1,5.9%,2026,5.57
EN29A,4.5% Endo Finance plc 2029,NaN,NaN,NaN,95.00,"50,000",1,4.5%,2029,5.47
PH28A,4.15% Phoenicia Finance Company plc 2028,1.0,"10,000",92.70,94.75,"11,700",1,4.15%,2028,5.34


In [19]:
total_trades = list_table[1].set_index('Symbol Code')
total_trades

,Volume Traded,Value Traded,Trades,High Price,Low Price,Open Price,Closing Price,Change
Symbol Code,,,,,,,,
G33A,"3,600","3,700.80",2,102.800,102.800,102.800,102.800,-0.140


In [20]:
df = pd.merge(trades,offers[['Best Offer Price','Best Offer Volume']], left_index=True, right_index=True, how='left')

In [21]:
df2 = pd.merge(df, total_trades[['Closing Price']], left_index=True, right_index=True, how='left')

In [22]:
df2

,Security Name,Best Bid Count,Best Bid Volume,Best Bid Price,Best Offer Price_x,Best Offer Volume_x,Best Offer Count,Best Offer Price_y,Best Offer Volume_y,Closing Price
Symbol Code,,,,,,,,,,
HSB,HSBC Bank Malta plc,1,"3,156",0.750,0.770,"63,582",2,NaN,NaN,NaN
MIA,Malta International Airport plc,1,"3,410",5.850,5.900,"14,432",5,NaN,NaN,NaN


In [23]:
offers_price = pd.merge(offers, total_trades[['Closing Price']], left_index=True, right_index=True, how='left')
offers_price

,Security Name,Best Bid Count,Best Bid Volume,Best Bid Price,Best Offer Price,Best Offer Volume,Best Offer Count,Closing Price
Symbol Code,,,,,,,,
GP25A,3.7% GAP Group plc 2025,-,-,-,99.990,"20,000",1,NaN
MB29A,4% MeDirect Bank Malta plc 2029,-,-,-,99.990,"25,000",1,NaN
MR26A,4% Merkanti Holding plc 2026,1,"10,000",94.000,99.990,"2,500",1,NaN
LS31A,4% LifeStar Insurance plc 2031,-,-,-,99.900,"30,000",1,NaN
BX27A,3.75% Bortex Group Finance plc 2027,-,-,-,99.750,"50,000",2,NaN
IH26B,4% International Hotel Investments plc 2026,1,"10,000",93.010,99.500,"20,000",1,NaN
TI27A,3.75% Tumas Investment plc 2027,-,-,-,99.500,"3,100",1,NaN
AX29A,3.75% AX Group plc 2029,-,-,-,99.500,"4,300",1,NaN
BR31A,3.9% Browns Pharma Holdings plc 2031,-,-,-,99.500,"5,000",1,NaN


In [24]:
offers_price = offers_price.replace('-', np.nan)
offers_price['Difference'] = offers_price['Best Offer Price'].astype(float)-offers_price['Closing Price'].astype(float)
offers_price['Difference'] = offers_price['Difference'].round(2)
offers_price

,Security Name,Best Bid Count,Best Bid Volume,Best Bid Price,Best Offer Price,Best Offer Volume,Best Offer Count,Closing Price,Difference
Symbol Code,,,,,,,,,
GP25A,3.7% GAP Group plc 2025,NaN,NaN,NaN,99.990,"20,000",1,NaN,NaN
MB29A,4% MeDirect Bank Malta plc 2029,NaN,NaN,NaN,99.990,"25,000",1,NaN,NaN
MR26A,4% Merkanti Holding plc 2026,1,"10,000",94.000,99.990,"2,500",1,NaN,NaN
LS31A,4% LifeStar Insurance plc 2031,NaN,NaN,NaN,99.900,"30,000",1,NaN,NaN
BX27A,3.75% Bortex Group Finance plc 2027,NaN,NaN,NaN,99.750,"50,000",2,NaN,NaN
IH26B,4% International Hotel Investments plc 2026,1,"10,000",93.010,99.500,"20,000",1,NaN,NaN
TI27A,3.75% Tumas Investment plc 2027,NaN,NaN,NaN,99.500,"3,100",1,NaN,NaN
AX29A,3.75% AX Group plc 2029,NaN,NaN,NaN,99.500,"4,300",1,NaN,NaN
BR31A,3.9% Browns Pharma Holdings plc 2031,NaN,NaN,NaN,99.500,"5,000",1,NaN,NaN


In [14]:
def is_offer_valid(temp):
    if  temp < 0.5:
        return 'background-color: green'
    if  temp > 1:
        return 'background-color: red'
offers_price = offers_price.sort_values(by=['Best Offer Price'], ascending=False)
s = offers_price.style.applymap(is_offer_valid, subset=['Difference'])
s

,Security Name,Best Bid Count,Best Bid Volume,Best Bid Price,Best Offer Price,Best Offer Volume,Best Offer Count,Closing Price,Difference
Symbol Code,,,,,,,,,
G36A,2.5% Treasury 2036,1,"4,500",85.210,99.000,"80,000",1,nan,nan
G41A,2.4% Treasury 2041,2,"21,500",91.000,99.000,"40,000",1,nan,nan
G40A,3% Treasury 2040,1,"4,000",86.000,98.500,"16,900",2,nan,nan
G39A,2.1% Treasury 2039,nan,nan,nan,95.490,"5,000",1,nan,nan
G51A,1.8% Treasury 2051,nan,nan,nan,95.000,"25,000",1,nan,nan
G37A,1.2% Treasury 2037,nan,nan,nan,93.200,"6,200",1,nan,nan
G30A,5.25% Treasury 2030,1,"100,000",100.000,118.000,"48,800",1,nan,nan
G31A,5.2% Treasury 2031,1,"75,000",100.000,113.000,"8,100",1,nan,nan
G32B,4.45% Treasury 2032,nan,nan,nan,110.410,"10,000",1,nan,nan
